In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd 'drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project/'
# %pwd

/content/drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project


In [ ]:
!pip install textstat
!pip install pyLDAvis

In [ ]:
# Imports
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import textstat

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import numpy as np

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium 
from folium import plugins 

import re


In [ ]:

#utility functions:
def plot_readability(a,b,title,bins=0.1,colors=['#3A4750', '#F64E8B']):
  trace1 = ff.create_distplot([a,b], [" Real disaster tweets","Not real disaster tweets"], bin_size=bins, colors=colors, show_rug=False)
  
  trace1['layout'].update(title=title)
  py.iplot(trace1, filename='Distplot')
  table_data= [["Statistical Measures"," Not real disaster tweets","real disaster tweets"], ["Mean",mean(a),mean(b)], ["Standard Deviation",pstdev(a),pstdev(b)],
               ["Variance",pvariance(a),pvariance(b)],
               ["Median",median(a),median(b)],
               ["Maximum value",max(a),max(b)],
               ["Minimum value",min(a),min(b)]]
  trace2 = ff.create_table(table_data)
  py.iplot(trace2, filename='Table')

punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens


def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def removeurl(raw_text):
  clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', raw_text, flags=re.MULTILINE)
  return clean_text

### Import Dataset


In [ ]:
# Importing cleaned data
train_df = pd.read_csv('data/preprocessed_train.csv')
test_df = pd.read_csv('data/preprocessed_test.csv')
sub_df = pd.read_csv('data/sample_submission.csv')


In [ ]:
train_df.head()

,Unnamed: 0,id,keyword,location,text,target,word_count,unique_word_count,stopword_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,text_cleaned,target_relabeled
0,0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1,13,13,5,0,4.384615,69,1,1,0,Our Deeds are the Reason of this # earthquake...,1
1,1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1,7,7,0,0,4.571429,38,1,0,0,Forest fire near La Ronge Sask . Canada,1
2,2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1,22,20,9,0,5.090909,133,3,0,0,All residents asked to ' shelter in place ' ...,1
3,3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,0,7.125000,65,2,1,0,"13,000 people receive # wildfires evacuation ...",1
4,4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,6,0,4.500000,88,2,2,0,Just got sent this photo from Ruby # Alaska a...,1


### Topic Modeling

In [ ]:
notreal_text = train_df[train_df["target"] == 0]['text_cleaned'].apply(spacy_tokenizer)
real_text = train_df[train_df["target"] == 1]['text_cleaned'].apply(spacy_tokenizer)
#count vectorization
vectorizer_notreal = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
notreal_vectorized = vectorizer_notreal.fit_transform(notreal_text)
vectorizer_real = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
real_vectorized = vectorizer_real.fit_transform(real_text)

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid 

#### Latent Dirichlet Allocation (LDA)

Latent Dirichlet Allocation (LDA) and LSA are based on the same underlying assumptions: the distributional hypothesis, (i.e. similar topics make use of similar words) and the statistical mixture hypothesis (i.e. documents talk about several topics) for which a statistical distribution can be determined. The purpose of LDA is mapping each document in our corpus to a set of topics which covers a good deal of the words in the document.

What LDA does in order to map the documents to a list of topics is assign topics to arrangements of words, e.g. n-grams such as best player for a topic related to sports. This stems from the assumption that documents are written with arrangements of words and that those arrangements determine topics.LDA ignores syntactic information and treats documents as bags of words. It also assumes that all words in the document can be assigned a probability of belonging to a topic. That said, the goal of LDA is to determine the mixture of topics that a document contains.

In other words, LDA assumes that topics and documents look like this:

In [ ]:
# Latent Dirichlet Allocation Model
lda_notreal = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
notreal_lda = lda_notreal.fit_transform(notreal_vectorized)
lda_real = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
real_lda = lda_real.fit_transform(real_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [ ]:
def selected_topics(model, vectorizer, top_n=10):
  for idx, topic in enumerate(model.components_):
    print("Topic %d:" % (idx))
    print([(vectorizer.get_feature_names()[i], topic[i])
    for i in topic.argsort()[:-top_n - 1:-1]]) 

In [ ]:
print("Not real disaster tweets LDA Model:")
selected_topics(lda_notreal, vectorizer_notreal)

Not real disaster tweets LDA Model:
Topic 0:
[('got', 91.84839846112526), ('wreck', 72.60816266378731), ('news', 71.18818344053022), ('right', 39.081538127403334), ('face', 38.07668880676441), ('loud', 36.8237130013706), ('new', 36.31661169582576), ('home', 35.78529842673892), ('coming', 34.44679910121407), ('bang', 33.03400258849955)]
Topic 1:
[('love', 102.31561606515294), ('good', 70.36581058088737), ('want', 66.352652760283), ('man', 62.12281961681981), ('lol', 50.551167360010126), ('great', 50.36674247338842), ('screaming', 45.82631605620366), ('smoke', 45.45700383815489), ('year', 40.06683398463897), ('school', 39.07012216681301)]
Topic 2:
[('reddit', 64.9687716678408), ('content', 55.82728563229023), ('trapped', 36.72069881243054), ('death', 35.5222172948022), ('quarantine', 34.238074091393244), ('hope', 31.58621559143264), ('thank', 29.605661423713716), ('offensive', 29.56959979045907), ('words', 27.11566626358977), ('traumatised', 25.69610306822761)]
Topic 3:
[('day', 87.62743

In [ ]:
print("Real disaster tweets LDA Model:")
selected_topics(lda_real, vectorizer_real)

Real disaster tweets LDA Model:
Topic 0:
[('confirmed', 72.19585253268673), ('investigators', 70.46883847110892), ('nuclear', 69.93558409959708), ('fatal', 56.134067860409), ('new', 53.325347013522354), ('forest', 48.30770213930744), ('wild', 46.85184705760967), ('near', 44.96582701941469), ('today', 44.05194244015352), ('watch', 41.40187717456447)]
Topic 1:
[('people', 76.81754099683697), ('train', 73.84804231209777), ('weather', 71.14551693081819), ('water', 54.405232892388874), ('home', 50.084476676794175), ('time', 42.70710640362133), ('live', 38.48558254496383), ('violent', 37.91415460488895), ('officer', 37.88419437831904), ('survivors', 34.963264196024255)]
Topic 2:
[('families', 116.0295289529817), ('homes', 61.52219506963639), ('rescuers', 39.671135227101345), ('area', 36.19330035142749), ('report', 34.02040920524111), ('tornado', 32.90469158621399), ('collapse', 31.566150294196216), ('refugees', 28.03838835942648), ('going', 26.671036284923144), ('weapons', 26.66115762966847)

##### Visualizing LDA results of not a real disaster tweets with pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_notreal, notreal_vectorized, vectorizer_notreal, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      32.420952   8.115230       1        1  12.174195
0       0.648955  71.707100       2        1  10.721154
7      71.019684   9.971006       3        1  10.585595
8       3.785981  33.185135       4        1  10.283793
4     -42.666500  -9.222070       5        1  10.215431
6      -3.621689 -43.526516       6        1   9.957067
5      -3.632402  -4.171708       7        1   9.437721
9     -36.584400  35.804413       8        1   9.019823
3      44.391380  50.615383       9        1   9.007960
2      40.235809 -31.591665      10        1   8.597261, topic_info=             Term        Freq       Total Category  logprob  loglift
720          like  252.000000  252.000000  Default  30.0000  30.0000
1307        video   88.000000   88.000000  Default  29.0000  29.0000
299           day   83.000000   83.000000  Default  28.0000  28.0000
521           got   86.000000   86.000000  Default  27.0000  27.0000
753          love   90.000000   90.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
216         chile   17.098468   18.581990  Topic10  -4.6034   2.3705
797        miners   18.381082   21.119378  Topic10  -4.5311   2.3149
960   quarantined   21.360078   26.472649  Topic10  -4.3809   2.2391
1263  traumatised   23.453882   40.743751  Topic10  -4.2873   1.9015
850       nuclear   20.496298   33.159802  Topic10  -4.4221   1.9726

[395 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        6  0.969729  actually
12        6  0.973095     added
28       10  0.952404   america
30        7  0.968486   ancient
33        7  0.169721    animal
...     ...       ...       ...
1391      1  0.967954      year
1392      3  0.986379     years
1393      1  0.949427       yes
1396      6  0.986664   youtube
1397      3  0.979772      zone

[466 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 8, 9, 5, 7, 6, 10, 4, 3])

##### Visualizing LDA results of a real disaster tweets with pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_real, real_vectorized, vectorizer_real, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  11.189724
0       0.648955  71.707100       2        1  11.095168
4      71.019684   9.971006       3        1  10.796635
5       3.785981  33.185135       4        1  10.709604
1     -42.666500  -9.222070       5        1  10.664516
3      -3.621689 -43.526516       6        1   9.834985
9      -3.632402  -4.171708       7        1   9.479658
7     -36.584400  35.804413       8        1   9.234937
2      44.391380  50.615383       9        1   8.921822
8      40.235809 -31.591665      10        1   8.072951, topic_info=            Term        Freq       Total Category  logprob  loglift
635     malaysia  185.000000  185.000000  Default  30.0000  30.0000
721         news  159.000000  159.000000  Default  29.0000  29.0000
295     disaster  123.000000  123.000000  Default  28.0000  28.0000
1027     suicide  127.000000  127.000000  Default  27.0000  27.0000
391       flight  127.000000  127.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1115     tsunami   15.185879   16.424673  Topic10  -4.5990   2.4382
137    buildings   42.096948   52.960487  Topic10  -3.5794   2.2871
242        crews   19.755128   23.108119  Topic10  -4.3360   2.3599
388       flames   18.108174   25.668344  Topic10  -4.4231   2.1678
148   california   17.566473  109.050014  Topic10  -4.4534   0.6908

[416 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         3  0.972230       abc
4         2  0.020607  accident
4         6  0.185462  accident
4         7  0.741849  accident
4         8  0.020607  accident
...     ...       ...       ...
1211      4  0.991863      year
1212      1  0.696560     years
1212      2  0.044939     years
1212      5  0.247166     years
1217      6  0.972895   youtube

[475 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 5, 6, 2, 4, 10, 8, 3, 9])

#### TFIDF Vectorization

In [ ]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit_transform(train_df['text_cleaned'].values.tolist() + test_df['text_cleaned'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text_cleaned'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text_cleaned'].values.tolist())

#### Building Classification models

##### 1. Logistic Regression

In [ ]:
train_y = train_df.target.values

def runModel(train_X, train_y, test_X, test_y, test_X2):
  model = linear_model.LogisticRegression(C=5., solver='sag')
  model.fit(train_X, train_y)
  pred_test_y = model.predict_proba(test_X)[:,1]
  pred_test_y2 = model.predict_proba(test_X2)[:,1]
  return pred_test_y, pred_test_y2, model

print("Building model.")

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0]])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=8)
for dev_index, val_index in kf.split(train_df):
  dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
  dev_y, val_y = train_y[dev_index], train_y[val_index]
  pred_val_y, pred_test_y, model = runModel(dev_X, dev_y, val_X, val_y, test_tfidf)
  pred_full_test = pred_full_test + pred_test_y
  pred_train[val_index] = pred_val_y
  cv_scores.append(metrics.log_loss(val_y, pred_val_y))
  break

Building model.


In [ ]:
from tqdm import tqdm

def threshold_search(y_true, y_proba):
  #reference: https://www.kaggle.com/hung96ad/pytorch-starter
  best_threshold = 0
  best_score = 0
  for threshold in tqdm([i * 0.001 for i in range(1000)]):
    score = metrics.f1_score(y_true=y_true, y_pred=y_proba > threshold)
    if score > best_score:
      best_threshold = threshold
      best_score = score
      search_result = {'threshold': best_threshold, 'f1': best_score}
      return search_result
      
search_result = threshold_search(val_y, pred_val_y)
search_result

  0%|          | 0/1000 [00:00<?, ?it/s]


{'f1': 0.6065873741994511, 'threshold': 0.0}

In [ ]:
print("F1 score at threshold {0} is {1}".format(0.381, metrics.f1_score(val_y, (pred_val_y>0.381).astype(int))))
print("Precision at threshold {0} is {1}".format(0.381, metrics.precision_score(val_y, (pred_val_y>0.381).astype(int))))
print("recall score at threshold {0} is {1}".format(0.381, metrics.recall_score(val_y, (pred_val_y>0.381).astype(int))))

F1 score at threshold 0.381 is 0.7624720774385705
Precision at threshold 0.381 is 0.7551622418879056
recall score at threshold 0.381 is 0.7699248120300752


In [38]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

def getClassifierObj(classifier, rs):
    '''Functio to get the classifier object
    '''
    if classifier == 'XGBoost':
        from xgboost import XGBClassifier
        classifier = XGBClassifier()
    
    elif classifier == 'LogisticRegression':
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = rs)
    
    elif classifier == 'KNN':
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    elif classifier == 'SVM':
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = rs)
        
    elif classifier == 'Kernel SVM':
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = rs)
        
    elif classifier == 'NB':
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        
    elif classifier == 'DecisionTree':
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = rs)
        
    elif classifier == 'RandomForest':
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = rs)
        
    return classifier
            
def fitAndPredict(estimator,X_train,y_train,X_test):
    '''Function to fit and predict
    '''
    estimator.fit(X_train,y_train)
    predictions = estimator.predict(X_test)
    return predictions

def getModelAccuracy(y_test, y_pred):
    '''Function to get model accuracy'''
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test, y_pred)*100

def getFinalPredictions(X_train,X_test, y_train, y_test):
    ''' Function to get classifier with highest accuracy on the data
    '''
    classifiers = ['LogisticRegression','KNN', 'Kernel SVM','DecisionTree','RandomForest','XGBoost']
    accuracy = 0
    best_predictions = ''
    bestClassifierName = ''
    classifier = ''
    
    for classifierName in classifiers:
        print('Evaluation started for ', classifierName)
        classifier = getClassifierObj(classifierName,1)
        y_pred = fitAndPredict(classifier,X_train,y_train,X_test)
        
        classifierAccuracy = getModelAccuracy(y_test,y_pred)
        print('Average accuracy of {} is {:.2f}%'.format(classifierName,classifierAccuracy))
        if classifierAccuracy > accuracy :
            accuracy = classifierAccuracy
            bestClassifierName = classifierName
            best_predictions = y_pred
            bestClassifier = classifier
            
            
    print('Classifier with highest accuracy is {}'.format(bestClassifierName))
    return best_predictions, accuracy, classifier      

def classify(X_train,X_test, y_train, y_test):
    final_pred, final_accuracy, classifier = getFinalPredictions(X_train,X_test, y_train, y_test)        
    cm = confusion_matrix(y_test, final_pred)
    print(cm)
    return classifier

In [39]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

train_y = train_df.target.values
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_y, random_state=8, test_size = 0.2)
bestClassifier = classify(X_train, X_test, y_train, y_test)

Evaluation started for  LogisticRegression
Average accuracy of LogisticRegression is 74.98%
Evaluation started for  KNN
Average accuracy of KNN is 75.38%
Evaluation started for  Kernel SVM
Average accuracy of Kernel SVM is 72.75%
Evaluation started for  DecisionTree
Average accuracy of DecisionTree is 71.63%
Evaluation started for  RandomForest
Average accuracy of RandomForest is 75.38%
Evaluation started for  XGBoost
Average accuracy of XGBoost is 70.26%
Classifier with highest accuracy is KNN
[[784  76]
 [299 364]]
